<a href="https://colab.research.google.com/github/Abhijith-Nagarajan/PadhAI_Implementations/blob/main/PyTorch_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np

In [ ]:
import time

#### Numpy vs Torch

In [ ]:
t1_cpu = torch.rand(10000,10000)
t2_cpu = torch.rand(10000,10000)
n1 = np.random.rand(10000,10000)
n2 = np.random.rand(10000,10000)

In [ ]:
%timeit
torch.matmul(t1_cpu,t2_cpu)

tensor([[2499.9170, 2493.0967, 2480.2927,  ..., 2465.6812, 2514.3569,
         2483.8689],
        [2540.5813, 2527.1001, 2526.7402,  ..., 2506.1538, 2546.0852,
         2532.7439],
        [2540.2122, 2548.6917, 2546.8420,  ..., 2508.9221, 2566.0166,
         2555.8655],
        ...,
        [2504.7231, 2474.8550, 2495.3953,  ..., 2460.7991, 2495.2456,
         2494.7983],
        [2504.6807, 2530.6089, 2494.1211,  ..., 2480.5576, 2524.7429,
         2513.0647],
        [2523.5400, 2519.4895, 2523.3132,  ..., 2509.4067, 2548.7229,
         2529.1426]])

In [ ]:
%timeit
np.matmul(n1,n2)

array([[2494.48525009, 2510.9557902 , 2501.27198668, ..., 2542.19756653,
        2529.59649212, 2517.05596801],
       [2492.04236243, 2532.09265513, 2487.90038279, ..., 2531.46015206,
        2544.00345436, 2505.05793283],
       [2460.46887452, 2451.59334611, 2453.55103164, ..., 2483.40292216,
        2486.46785132, 2457.29986758],
       ...,
       [2489.53173066, 2490.35439993, 2484.20228343, ..., 2522.37800905,
        2524.27809434, 2484.08090673],
       [2468.54952269, 2504.44917688, 2495.78776398, ..., 2529.1117802 ,
        2507.81712966, 2502.10996621],
       [2483.60196609, 2490.52045157, 2486.87966377, ..., 2510.76372723,
        2529.8636999 , 2490.42027807]])

#### Torch with GPU

In [ ]:
# Connecting to a GPU
torch.cuda.device_count()

1

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
cuda_obj = torch.device('cuda:0')

In [ ]:
start = time.time()
for i in range(10):
    t1 = torch.rand([10000,10000],device=cuda_obj)
    t2 = torch.rand([10000,10000],device=cuda_obj)
    print(t1+t2)
    print(torch.matmul(t1,t2))
end = time.time()
print(end-start)

tensor([[0.9541, 0.4266, 1.3273,  ..., 1.0642, 0.7480, 0.4148],
        [0.5190, 0.7814, 1.0621,  ..., 0.5479, 1.8589, 0.8138],
        [0.7028, 0.9372, 0.9956,  ..., 1.2969, 1.1910, 0.9372],
        ...,
        [1.5738, 1.3562, 1.8152,  ..., 0.4472, 1.1565, 0.3244],
        [0.7644, 0.5761, 1.0035,  ..., 0.6719, 1.7868, 1.1175],
        [1.7526, 1.3008, 0.4900,  ..., 1.4067, 0.4921, 0.3142]],
       device='cuda:0')
tensor([[2494.8682, 2503.8503, 2505.1370,  ..., 2531.9072, 2462.0247,
         2509.9929],
        [2489.0195, 2483.3682, 2482.3821,  ..., 2508.5359, 2460.1628,
         2473.5110],
        [2484.2312, 2492.6824, 2493.3616,  ..., 2522.0227, 2457.5522,
         2473.9587],
        ...,
        [2487.5317, 2502.9595, 2516.6506,  ..., 2535.4502, 2472.7336,
         2493.9841],
        [2484.7292, 2510.6187, 2482.5383,  ..., 2515.3533, 2462.7520,
         2480.1768],
        [2479.7473, 2501.5134, 2490.0371,  ..., 2514.5894, 2476.7803,
         2508.0046]], device='cuda:0')
t